In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/emnlp-test-enbodo-2025/en-bodo.txt
/kaggle/input/engbodo/English-Bodo Traning Data 2025.xlsx


In [3]:
!pip install -q transformers datasets accelerate bitsandbytes peft evaluate sacrebleu
!pip install -U bitsandbytes
!pip install datasets pandas sacremoses fasttext sentence-transformers sentencepiece scikit-learn
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
!pip install spacy indic-transliteration
!python -m spacy download en_core_web_sm 
!pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install indic-transliteration

In [5]:
import torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq)
from transformers import BitsAndBytesConfig
from peft import LoraConfig, TaskType, get_peft_model
import evaluate 

2025-06-15 08:18:42.579243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749975522.816667      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749975522.883096      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
import pandas as pd

file_path = "/kaggle/input/engbodo/English-Bodo Traning Data 2025.xlsx"
df = pd.read_excel(file_path)

csv_path = "/kaggle/working/english_bodo_training_data_2025.csv"
df.to_csv(csv_path, index=False)

print(f"CSV file saved to: {csv_path}") 

CSV file saved to: /kaggle/working/english_bodo_training_data_2025.csv


In [12]:
import pandas as pd
df = df.drop(columns=["Unnamed: 2", "Unnamed: 3"], errors='ignore')  # returns a new DataFrame
df.to_csv("bodo.csv", index=False)

In [13]:
df

,English,Bodo
0,The Indian independence movement was a signifi...,भारतनि उदांस्रि सोमावसारनाया भारतनि जारिमिनाव ...
1,Mahatma Gandhi emerged as the leader of the In...,महात्मा गान्धीआ भारतारि उदांस्रि सोमावसारनायनि...
2,"The Salt March of 1930, led by Mahatma Gandhi,...",महात्मा गान्धीनि दैदेन्नायाव 1930 मायथाइनि संख...
3,"The Quit India Movement, launched in 1942, was...",1942 मायथाइयाव जागायजेन्नाय भारत नागार सोमावसा...
4,"Jawaharlal Nehru, the first Prime Minister of ...",उदां भारतनि गिबि गाहाय मन्थ्रि जवाहरलाल नेहरुआ...
...,...,...
15210,The Assam State Textbook Production and Public...,आसाम रायजो फराय बिजाब दिहुन्नाय आरो पाब्लिकेसन...
15211,The state's economic development is supported ...,रायजोनि रांखान्थियारि जौगानाया बिनि मिथिंगायार...
15212,The government has launched various initiative...,सोरखारा आबहावा संरैखा खालामनाय आरो अरायथा जौगा...
15213,"Assam's renewable energy sector is growing, wi...","सानखरां, बार आरो दैमोब्लिब मावथांखिफोराव रां थ..."


In [14]:

dataset = load_dataset(
    'csv', 
    data_files="bodo.csv", 
    split="train"
)

subset = dataset.shuffle(seed=42).select(range(15215 ))

dataset_split = subset.train_test_split(test_size=0.1) 
train_dataset = dataset_split['train']
test_dataset = dataset_split['test']

print(f"Train examples: {len(train_dataset)}, Test examples: {len(test_dataset)}")

Generating train split: 0 examples [00:00, ? examples/s]

Train examples: 13693, Test examples: 1522


In [15]:
def normalize_text(text):
    if text is None:
        return ""
    
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x20-\x7E\u0900-\u097F\u0980-\u09FF\u0A80-\u0AFF\u1C00-\u1C4F]', '', text)
    text = text.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")
    
    return text
def filter_missing(example):
    return example["English"] is not None and example["Bodo"] is not None

train_dataset = train_dataset.filter(filter_missing)
test_dataset = test_dataset.filter(filter_missing)
train_dataset = train_dataset.map(preprocess_function)
test_dataset = test_dataset.map(preprocess_function)


Filter:   0%|          | 0/13693 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1522 [00:00<?, ? examples/s]

NameError: name 'preprocess_function' is not defined

In [16]:
train_dataset

Dataset({
    features: ['English', 'Bodo'],
    num_rows: 13692
})

In [17]:
import pandas as pd

data = pd.DataFrame(train_dataset[:10])  # Load first 10 rows
data


,English,Bodo
0,"Nature’s cycles of growth, decay, and renewal ...","मिथिंगानि जौगानाय, जोबस्रांनाय आरो गोदान खालाम..."
1,China and the United States have the highest n...,चीन आरो जुथाइ रायजो आमेरिकायाव बुहुमाव बयनिख्र...
2,E-commerce transformed retail by allowing busi...,ई-कमर्सआ फालांगिफोरखौ मुलुगनाङै बायगिरिफोरसिम ...
3,He is of the opinion that filmmakers only targ...,बिथांनि साननाया बेनो दि सावथुन बानायगिरिफोरा ग...
4,Representations of binary tree in memory.,मेम'रिआव बाइनेरि ट्रीनि दिन्थिफुंनाय।
5,The celebration of Bwisagu is a time of joy wh...,बैसागुनि फालिथाया रंजानायनि मोनसे सम जेराव नखर...
6,The Bodo people have a deep connection to thei...,बर' सुबुंफोरा गावसोरनि हाजों गोथौ सोमोन्दो दं ...
7,Delhi LG’s Proposal Return: Delhi LG Vinai Kum...,दिल्ली एलजीनि प्रप'जल रिटार्न: दिल्लीनि एल.जि....
8,Cultural Exchange: Sharing of cultures through...,"हारिमुआरि सोलाय-सोल': सावथुन, देंखो आरो बिजोंन..."
9,Modi’s government faces opposition challenges ...,म'दिनि सोरखारा रांखान्थियारि बिथांखिफोरखौ लाना...


In [18]:
from transformers import AutoTokenizer

model_name = "ai4bharat/indictrans2-en-indic-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_examples(examples):
    inputs = [f"eng_Latn brx_Deva {en}" for en in examples["English"]]
    targets = examples["Bodo"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="longest",
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            padding="longest",
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
  
# Then proceed with dataset processing
train_dataset = train_dataset.map(preprocess_examples, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_examples, batched=True, remove_columns=test_dataset.column_names)

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.04k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

...

model.TGT:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Map:   0%|          | 0/13692 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1522 [00:00<?, ? examples/s]

In [19]:
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch
import torch.nn as nn

bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "ai4bharat/indictrans2-en-indic-1B",
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    trust_remote_code=True,  
    #device_map="auto"        
)

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Set pad token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Your input sentence
english_text = "The weather is very good today."

# Format input as per IndicTrans2 requirements
formatted_input = f"eng_Latn brx_Deva {english_text}"

# Tokenize
inputs = tokenizer(
    formatted_input,
    return_tensors="pt",
    padding="longest",
    truncation=True
)

# Generate translation
output = model.generate(
    **inputs,
    max_length=128,
    num_beams=5,
    early_stopping=True
)

# Decode output
bodo_translation = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

print("English:", english_text)
print("Bodo:", bodo_translation)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [21]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=64,
    lora_alpha=128,
    target_modules=["q_proj", "v_proj","k_proj","dense"], 
    lora_dropout=0.1,
    bias="none",
    use_dora=True
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() 

trainable params: 21,399,552 || all params: 1,136,943,104 || trainable%: 1.8822


In [22]:
device = torch.device("cuda")

model=model.to(device)

In [23]:
data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding="longest",
        pad_to_multiple_of=8, 
        label_pad_token_id=-100
)

In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [25]:
bleu_metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# def transliterate_predictions(preds):
#     # Transliterate from Devanagari to Bengali script
#     return [transliterate(pred, sanscript.DEVANAGARI, sanscript.BENGALI) for pred in preds]

def compute_metrics(eval_preds):
    preds, labels = eval_preds.predictions, eval_preds.label_ids
    
    if isinstance(preds, tuple):
        preds = preds[0]
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Transliterate predicted Devanagari output to Bengali script
    # decoded_preds = transliterate_predictions(decoded_preds)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [26]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=3e-5, 
    warmup_steps=200, # for entire dataset 1000
    lr_scheduler_type="cosine", 
    logging_steps=50, # for entire dataset 500
    eval_strategy="epoch",
    save_strategy="epoch",
   # save_steps=1000,
    save_total_limit=2,
    fp16=True,
    predict_with_generate=True,
    report_to="none",
    optim="adamw_torch_fused",
    adam_beta1=0.9,
    adam_beta2=0.98,
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
model.config.use_cache = False

/tmp/ipykernel_35/3633880997.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [27]:
trainer.train()   

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Bleu
1,7.583100,7.431449,17.591270


TrainOutput(global_step=856, training_loss=8.230269053271998, metrics={'train_runtime': 1903.7445, 'train_samples_per_second': 7.192, 'train_steps_per_second': 0.45, 'total_flos': 6052875643256832.0, 'train_loss': 8.230269053271998, 'epoch': 1.0})

In [28]:
predict_results = trainer.predict(test_dataset)
predictions = predict_results.predictions
labels = predict_results.label_ids

# If predictions is tuple, take first element
if isinstance(predictions, tuple):
    predictions = predictions[0]

# If predictions are logits (batch_size, seq_len, vocab_size), convert to token IDs
if predictions.ndim == 3:
    predictions = np.argmax(predictions, axis=-1)

# Replace -100 in labels (ignored tokens) with pad_token_id for decoding
labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

# Decode to text
decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# Transliterate predicted Devanagari output to Bengali script for Assamese
# decoded_preds = transliterate_predictions(decoded_preds)

# Postprocess (strip whitespace etc)
decoded_preds, processed_references = postprocess_text(decoded_preds, decoded_labels)

# Compute BLEU score
result = bleu_metric.compute(predictions=decoded_preds, references=processed_references)
bleu_score = result["score"]

print(f"SacreBLEU Score: {bleu_score:.4f}")
print(f"Detailed BLEU: {result}")

print("\nSample Actual vs Predicted Translations:")
for i in range(min(5, len(decoded_labels))):
    print(f"  Example {i+1}:")
    print(f"    Actual:    {decoded_labels[i]}")
    print(f"    Predicted: {decoded_preds[i]}\n")


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


SacreBLEU Score: 17.5913
Detailed BLEU: {'score': 17.59126996196025, 'counts': [8762, 4538, 2454, 1393], 'totals': [17794, 16272, 14752, 13233], 'precisions': [49.24131729796561, 27.888397246804328, 16.635032537960953, 10.526713519232223], 'bp': 0.7943826938525097, 'sys_len': 17794, 'ref_len': 21890}

Sample Actual vs Predicted Translations:
  Example 1:
    Actual:    पि. एम जानो लुबैनायाव जेबो गोरोन्थि गैयाः नायडु
    Predicted: पि जानो गोसोयाव जेबो गोरोन्थि गैयाः नायडु

  Example 2:
    Actual:    इउ.के. टेक्स अथ'रिटिआ भेट होफिन्नाय गोबावनि गेजेराव उफ्रा फुंखानि थाखाय रादाय होयो
    Predicted: " वेटखौ सुख'नायाव इउके टेक्स अथ'रिटिआ बांद्राय फुंखा होनो गनायथि होयो

  Example 3:
    Actual:    मोब्लिबारि कारफोरा जोरनो हानाय आरो खहागोनां हगारनायखौ बाङाय खालामनानै अट'म'टिभ दारिमिनखौ सोलायहोदोंमोन।
    Predicted: मोब्लिबजों सालायजानाय कारफोरा जोर होयो आरो खहा खालामग्रा बिदुं ओंखारनायखौ खम खालामो ।

  Example 4:
    Actual:    आमेरिकानि गाहाय नोगोरमाफोराव कभिड-19 टिका थुनायनि कभारेजआ 90 जौ

In [29]:
# Save final model and tokenizer
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/dict.SRC.json',
 './final_model/dict.TGT.json',
 './final_model/model.SRC',
 './final_model/model.TGT',
 './final_model/added_tokens.json')

In [30]:
import shutil

shutil.make_archive("/kaggle/working/final_model", 'zip', "./final_model")

'/kaggle/working/final_model.zip'

In [35]:
test_file_path = "/kaggle/input/emnlp-test-enbodo-2025/en-bodo.txt"

# Read English lines
english_sentences = []
with open(test_file_path, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) >= 1:
            english_sentences.append(parts[0])


In [36]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

pretrained_tokenizer_name = "ai4bharat/indictrans2-en-indic-1B"
model_path = "./final_model"

# Load tokenizer from original repo (with trust_remote_code)
tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer_name, trust_remote_code=True)

# Load fine-tuned model from local checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

model.eval().cuda()  # if GPU available


The repository for ai4bharat/indictrans2-en-indic-1B contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/ai4bharat/indictrans2-en-indic-1B.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


IndicTransForConditionalGeneration(
  (model): IndicTransModel(
    (encoder): IndicTransEncoder(
      (embed_tokens): Embedding(32322, 1024, padding_idx=1)
      (embed_positions): IndicTransSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-17): 18 x IndicTransEncoderLayer(
          (self_attn): IndicTransAttention(
            (k_proj): lora.Linear(
              (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=64, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=64, out_features=1024, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): ModuleDict(
        

In [ ]:
import torch

def batch_translate(sentences, model, tokenizer, batch_size=16, src_lang="eng_Latn", tgt_lang="brx_Deva"):
    all_outputs = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        inputs = [f"{src_lang} {tgt_lang} {text}" for text in batch]
        encoded = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **encoded,
                max_length=128,
                num_beams=5,
                early_stopping=True
            )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_outputs.extend(decoded)
    return all_outputs

# Generate predictions
assamese_predictions = batch_translate(english_sentences, model, tokenizer)


In [ ]:
# Save to myresult.txt
with open("myresult.txt", "w", encoding="utf-8") as f:
    for translation in assamese_predictions:
        f.write(translation.strip() + "\n")
